In [6]:
import pandas as pd
import numpy as np
from sklearn.metrics import roc_auc_score
import os
from datetime import datetime as dt

In [2]:
os.chdir(r"C:\Users\aman.khatri91425\Desktop\HomeCredit\UW\Udaan POS Analysis")

In [13]:
df = pd.read_csv(os.getcwd()+ r"\Data Files\loan_performance_udaan.csv")

In [14]:
df.head()

,SKP_CREDIT_CASE,POS_TYPE,TIME_CREATION_DATE,RISK_FPD10,RISK_AGRF10,RISK_FPD30,RISK_AGRF30,RISK_FSPD30,RISK_AGRF60,RISK_FSTPD30,...,RISK_FSTQPD30,RISK_AGRF120,RISK_FSTQPD60,RISK_AGRF150,POS_AUG20_GM,POS_AUG20_GM_WO_PRODUCT,SCORE_APSC_NOV18,CLIENT_EXI_3M,FLAG_1PCT_PHANTOM,NTC_FLAG
0,318014362,UDAAN,07-09-20,0,1,0,1,0,1,0,...,0,0,0,0,NaN,NaN,0.007830,N,0,NTC
1,318037792,UDAAN,07-09-20,0,1,0,1,0,1,0,...,0,0,0,0,NaN,NaN,0.009900,N,0,Others
2,318033501,UDAAN,07-09-20,0,1,0,1,0,1,0,...,0,0,0,0,NaN,NaN,0.004361,N,0,NTC
3,318031713,UDAAN,07-09-20,0,1,0,1,0,1,0,...,0,0,0,0,NaN,NaN,0.008086,N,0,NTC
4,318031874,UDAAN,07-09-20,0,1,0,1,0,1,0,...,0,0,0,0,NaN,NaN,0.009065,N,0,NTC


In [15]:
df["TIME_CREATION_DATE"] = pd.to_datetime(df["TIME_CREATION_DATE"],format = '%d-%m-%y')
df["MONTH"] = df["TIME_CREATION_DATE"].dt.strftime('%Y-%m')
df["MONTH"].value_counts().sort_index()

2020-09    1503
2020-10    2540
2020-11    4076
2020-12    2568
Name: MONTH, dtype: int64

In [16]:
df = df[df.TIME_CREATION_DATE>=dt(2020,10,15)]

# Gini: FPD10 Rate

In [18]:
score = 'POS_AUG20_GM'
df_temp = df.dropna(subset = [score])[df["RISK_AGRF10"]==1]
print("Score :", score, "\t\tObservables :", len(df_temp), "\t\tGini :", 2*roc_auc_score(df_temp["RISK_FPD10"],df_temp[score])-1)

score = 'POS_AUG20_GM_WO_PRODUCT'
df_temp = df.dropna(subset = [score])[df["RISK_AGRF10"]==1]
print("Score :", score, "\t\tObservables :", len(df_temp), "\t\tGini :", 2*roc_auc_score(df_temp["RISK_FPD10"],df_temp[score])-1)

score = 'SCORE_APSC_NOV18'
df_temp = df.dropna(subset = [score])[df["RISK_AGRF10"]==1]
print("Score :", score, "\t\tObservables :", len(df_temp), "\t\tGini :", 2*roc_auc_score(df_temp["RISK_FPD10"],df_temp[score])-1)


Score : POS_AUG20_GM 		Observables : 2544 		Gini : 0.15838481254807224
Score : POS_AUG20_GM_WO_PRODUCT 		Observables : 2531 		Gini : 0.15876180247908
Score : SCORE_APSC_NOV18 		Observables : 2544 		Gini : 0.0565408171586117


C:\Users\aman.khatri91425\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


# Pentile wise FPD10 rate

In [19]:
score = 'POS_AUG20_GM'
df_temp = df.dropna(subset = [score])[df["RISK_AGRF10"]==1]
df_temp["SCORE_BIN"] = pd.qcut(df_temp[score],q=5, precision = 3)
df_temp.head()
print(score)
df_temp.groupby("SCORE_BIN")["RISK_FPD10"].agg(['count','sum','mean']).rename(columns={'count':'Total_Case',
                                                                                                      'sum':'FPD10_Delinq',
                                                                                                      'mean':'FPD10_Rate'})

POS_AUG20_GM


,Total_Case,FPD10_Delinq,FPD10_Rate
SCORE_BIN,,,
"(0.0001299999999999999, 0.00922]",509,26,0.051081
"(0.00922, 0.0122]",509,37,0.072692
"(0.0122, 0.0158]",508,34,0.066929
"(0.0158, 0.0217]",509,42,0.082515
"(0.0217, 0.144]",509,54,0.106090


In [20]:
score = 'POS_AUG20_GM_WO_PRODUCT'
df_temp = df.dropna(subset = [score])[df["RISK_AGRF10"]==1]
df_temp["SCORE_BIN"] = pd.qcut(df_temp[score],q=5, precision = 3)
df_temp.head()
print(score)
df_temp.groupby("SCORE_BIN")["RISK_FPD10"].agg(['count','sum','mean']).rename(columns={'count':'Total_Case',
                                                                                                      'sum':'FPD10_Delinq',
                                                                                                      'mean':'FPD10_Rate'})

POS_AUG20_GM_WO_PRODUCT


C:\Users\aman.khatri91425\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


,Total_Case,FPD10_Delinq,FPD10_Rate
SCORE_BIN,,,
"(0.00025, 0.0118]",507,28,0.055227
"(0.0118, 0.0156]",506,34,0.067194
"(0.0156, 0.02]",506,38,0.075099
"(0.02, 0.0275]",506,32,0.063241
"(0.0275, 0.206]",506,59,0.116601


In [21]:
score = 'SCORE_APSC_NOV18'
df_temp = df.dropna(subset = [score])[df["RISK_AGRF10"]==1]
df_temp["SCORE_BIN"] = pd.qcut(df_temp[score],q=5, precision = 3)
df_temp.head()
print(score)
df_temp.groupby("SCORE_BIN")["RISK_FPD10"].agg(['count','sum','mean']).rename(columns={'count':'Total_Case',
                                                                                                      'sum':'FPD10_Delinq',
                                                                                                      'mean':'FPD10_Rate'})

SCORE_APSC_NOV18


,Total_Case,FPD10_Delinq,FPD10_Rate
SCORE_BIN,,,
"(0.0009599999999999999, 0.00599]",509,34,0.066798
"(0.00599, 0.00768]",509,33,0.064833
"(0.00768, 0.00943]",508,45,0.088583
"(0.00943, 0.0122]",509,39,0.076621
"(0.0122, 0.0515]",509,42,0.082515


# Bureau Found Not Found FPD30 Rate

In [22]:
df[df["RISK_AGRF10"]==1].groupby("NTC_FLAG")["RISK_FPD10"].agg(['count','sum','mean']).rename(columns={'count':'Total_Case',
                                                                                                      'sum':'FPD10_Delinq',
                                                                                                      'mean':'FPD10_Rate'})

,Total_Case,FPD10_Delinq,FPD10_Rate
NTC_FLAG,,,
NTC,1681,118,0.070196
Others,863,75,0.086906


In [23]:
score = 'POS_AUG20_GM'
df_temp = df.dropna(subset = [score])[(df["RISK_AGRF10"]==1)&(df["NTC_FLAG"]=='NTC')]
print("Score :", score, "\t\tObservables :", len(df_temp), "\t\tNTC Gini :", 2*roc_auc_score(df_temp["RISK_FPD10"],df_temp[score])-1)
df_temp = df.dropna(subset = [score])[(df["RISK_AGRF10"]==1)&(df["NTC_FLAG"]!='NTC')]
print("Score :", score, "\t\tObservables :", len(df_temp), "\t\tExi Gini :", 2*roc_auc_score(df_temp["RISK_FPD10"],df_temp[score])-1)


score = 'POS_AUG20_GM_WO_PRODUCT'
df_temp = df.dropna(subset = [score])[(df["RISK_AGRF10"]==1)&(df["NTC_FLAG"]=='NTC')]
print("Score :", score, "\t\tObservables :", len(df_temp), "\t\tNTC Gini :", 2*roc_auc_score(df_temp["RISK_FPD10"],df_temp[score])-1)
df_temp = df.dropna(subset = [score])[(df["RISK_AGRF10"]==1)&(df["NTC_FLAG"]!='NTC')]
print("Score :", score, "\t\tObservables :", len(df_temp), "\t\tExi Gini :", 2*roc_auc_score(df_temp["RISK_FPD10"],df_temp[score])-1)


score = 'SCORE_APSC_NOV18'
df_temp = df.dropna(subset = [score])[(df["RISK_AGRF10"]==1)&(df["NTC_FLAG"]=='NTC')]
print("Score :", score, "\t\tObservables :", len(df_temp), "\t\tNTC Gini :", 2*roc_auc_score(df_temp["RISK_FPD10"],df_temp[score])-1)
df_temp = df.dropna(subset = [score])[(df["RISK_AGRF10"]==1)&(df["NTC_FLAG"]!='NTC')]
print("Score :", score, "\t\tObservables :", len(df_temp), "\t\tExi Gini :", 2*roc_auc_score(df_temp["RISK_FPD10"],df_temp[score])-1)


Score : POS_AUG20_GM 		Observables : 1681 		NTC Gini : 0.13341900083498692
Score : POS_AUG20_GM 		Observables : 863 		Exi Gini : 0.18233502538071078
Score : POS_AUG20_GM_WO_PRODUCT 		Observables : 1675 		NTC Gini : 0.15022339696091658
Score : POS_AUG20_GM_WO_PRODUCT 		Observables : 856 		Exi Gini : 0.16162185232607773
Score : SCORE_APSC_NOV18 		Observables : 1681 		NTC Gini : 0.06608326013641741
Score : SCORE_APSC_NOV18 		Observables : 863 		Exi Gini : 0.057648054145516214


C:\Users\aman.khatri91425\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:9: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  if __name__ == '__main__':
C:\Users\aman.khatri91425\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:11: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  # This is added back by InteractiveShellApp.init_path()
